In [52]:
import numpy as np
import pandas as pd
import requests
import geopandas as gpd
from geopy.geocoders import Nominatim
import folium
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded
from shapely.geometry import Point
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
from rank_bm25 import BM25Okapi
from datetime import datetime


# NYC Parking Violation

### Datasets and Data Clean

In [30]:

violation_data_url = "https://data.cityofnewyork.us/resource/kvfd-bves.json"
params = {'$where': "issue_date <= '2023-12-31T23:59:59'", '$limit': 1000000}
response = requests.get(violation_data_url, params=params)
violation_data = response.json()
df_violation = pd.DataFrame(violation_data)


violation_codes_url = 'https://data.cityofnewyork.us/resource/ncbg-6agr.json'
response = requests.get(violation_codes_url)
violation_codes_data = response.json()
df_violation_codes = pd.DataFrame(violation_codes_data)

df_violation['violation_code'] = df_violation['violation_code'].astype(str)
df_violation_codes['code'] = df_violation_codes['code'].astype(str)

df1 = pd.merge(df_violation, df_violation_codes[['code', 'definition']], left_on='violation_code', right_on='code', how='left')
df1 = df1.drop(columns='code')
df1 = df1.rename(columns={'definition': 'code_definition'})


In [31]:
COLUMNS = [
    'summons_number',
    'plate_id',
    'registration_state',
    'plate_type',
    'issue_date',
    'violation_code',
    'vehicle_body_type',
    'vehicle_make',
    'issuing_agency',
    'street_name',
    'intersecting_street',
    'vehicle_expiration_date',
    'violation_time',
    'violation_county',
    'violation_description',
    'code_definition'
]

df = df1[COLUMNS]

In [32]:
df.duplicated().sum()

0

In [33]:
df.columns

Index(['summons_number', 'plate_id', 'registration_state', 'plate_type',
       'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make',
       'issuing_agency', 'street_name', 'intersecting_street',
       'vehicle_expiration_date', 'violation_time', 'violation_county',
       'violation_description', 'code_definition'],
      dtype='object')

In [34]:
df.isna().sum()

summons_number                  0
plate_id                        0
registration_state              0
plate_type                      0
issue_date                      0
violation_code                  0
vehicle_body_type            2706
vehicle_make                 2006
issuing_agency                  0
street_name                   111
intersecting_street        404751
vehicle_expiration_date         0
violation_time                 11
violation_county             1848
violation_description      462655
code_definition               181
dtype: int64

In [35]:
def convert_date(date_str):
    if date_str in ["88880088", "0"]:
        return np.nan
    else:
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')

def convert_time(time_str):
    if not isinstance(time_str, str):
        return time_str
    if time_str[-1] == 'P':
        hour = int(time_str[:2])
        if hour < 12:
            hour += 12
        return f"{hour:02d}:{time_str[2:4]}"
    elif time_str[-1] == 'A':
        hour = int(time_str[:2])
        return f"{hour:02d}:{time_str[2:4]}"
    else:
        return time_str

df['issue_date'] = pd.to_datetime(df['issue_date']).dt.date
df['vehicle_expiration_date'] = df['vehicle_expiration_date'].apply(convert_date)
df['violation_time'] = df['violation_time'].apply(convert_time)

/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3097254387.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['issue_date'] = pd.to_datetime(df['issue_date']).dt.date
/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3097254387.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vehicle_expiration_date'] = df['vehicle_expiration_date'].apply(convert_date)
/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3097254387.py:23: SettingWithCopyWarning: 
A va

In [9]:
df[['issue_date','vehicle_expiration_date','violation_time']]

,issue_date,vehicle_expiration_date,violation_time
0,2020-07-02,2022-05-10,12:59
1,2020-06-27,2020-09-09,09:40
2,2020-06-27,2022-03-06,09:40
3,2020-06-27,2021-05-06,09:40
4,2020-06-27,2022-05-24,09:40
...,...,...,...
999995,2020-07-28,NaT,15:06
999996,2020-07-28,NaT,15:06
999997,2020-07-28,NaT,15:06
999998,2020-07-28,NaT,15:06


In [36]:
df

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_name,intersecting_street,vehicle_expiration_date,violation_time,violation_county,violation_description,code_definition
0,1471497410,HZH8177,NY,PAS,2020-07-02,20,SUBN,NISSA,P,SHORE FRONT PKWY,NaN,2022-05-10,12:59,Q,NaN,NO PARKING-DAY/TIME LIMITS
1,1471497630,JCX5781,NY,PAS,2020-06-27,20,P-U,DODGE,P,ROCKAWAY BEACH BLVD,NaN,2020-09-09,09:40,Q,NaN,NO PARKING-DAY/TIME LIMITS
2,1471497641,HEK2391,NY,PAS,2020-06-27,20,SUBN,KIA,P,ROCKAWAY BEACH BLVD,NaN,2022-03-06,09:40,Q,NaN,NO PARKING-DAY/TIME LIMITS
3,1471497653,GWY9859,NY,PAS,2020-06-27,20,SUBN,JEEP,P,ROCKAWAY BEACH BLVD,NaN,2021-05-06,09:40,Q,NaN,NO PARKING-DAY/TIME LIMITS
4,1471497665,HEZ5501,NY,PAS,2020-06-27,20,SUBN,SUBAR,P,ROCKAWAY BEACH BLVD,NaN,2022-05-24,09:40,Q,NaN,NO PARKING-DAY/TIME LIMITS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,4700723865,HSC7218,NY,PAS,2020-07-28,36,4DSD,ACURA,V,EB NEPTUNE AVE @ W 5,TH ST,NaT,15:06,BK,PHTO SCHOOL ZN SPEED VIOLATION,PHTO SCHOOL ZN SPEED VIOLATION
999996,4700723889,HVN4751,NY,PAS,2020-07-28,36,SUBN,TOYOT,V,EB UNION TPKE @ KENT,ST,NaT,15:06,QN,PHTO SCHOOL ZN SPEED VIOLATION,PHTO SCHOOL ZN SPEED VIOLATION
999997,4700723890,JEC5912,NY,PAS,2020-07-28,36,4DSD,HONDA,V,SB BELL BLVD @ 18TH,AVE,NaT,15:06,QN,PHTO SCHOOL ZN SPEED VIOLATION,PHTO SCHOOL ZN SPEED VIOLATION
999998,4700723932,JRA7084,NY,PAS,2020-07-28,36,4DSD,ME/BE,V,NB KISSENA BLVD @ ME,LBOURNE AVE,NaT,15:06,QN,PHTO SCHOOL ZN SPEED VIOLATION,PHTO SCHOOL ZN SPEED VIOLATION


In [41]:
df['full_address'] = df['street_name'] + ', New York, NY'

geolocator = Nominatim(user_agent="geoapiExercises")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)


/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3792662942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_address'] = df['street_name'] + ', New York, NY'


In [46]:
def custom_score(row, query_date, query_violation_type):
    date_score = 1 / (np.abs((pd.to_datetime(row['issue_date']) - query_date).days) + 1)
    violation_type_score = 1 if row['violation_description'] == query_violation_type else 0
    return date_score + violation_type_score


In [47]:
def search_violations(query_date, query_violation_type):
    df['score'] = df.apply(custom_score, args=(query_date, query_violation_type), axis=1)
    return df.sort_values(by='score', ascending=False).head(10)


In [48]:
query_date = pd.to_datetime('2021-01-01')
query_violation_type = 'NO PARKING-DAY/TIME LIMITS'

print(search_violations(query_date, query_violation_type))


/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/2246792126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df.apply(custom_score, args=(query_date, query_violation_type), axis=1)


       summons_number plate_id registration_state plate_type  issue_date  \
415430     1474488717  T657840                 NY        OMT  2020-11-30   
972356     1462164158  JGW9563                 NY        PAS  2021-03-11   
413584     1472669654  HTC4835                 NY        PAS  2020-10-20   
414033     1477377451  2602524                 IN        PAS  2020-10-19   
972463     1472538353  KDE8968                 NY        PAS  2020-10-16   
972464     1472538407  JLV2422                 NY        PAS  2020-10-11   
972462     1472538341   IYTH74                 FL        PAS  2020-10-11   
2213       1455344539  JWB8295                 PA        PAS  2020-09-29   
398924     1475040659  7534639                 GV        PAS  2020-09-28   
1254       1439960483  JNB4368                 NY        PAS  2020-09-21   

       violation_code vehicle_body_type vehicle_make issuing_agency  \
415430             14              SUBN        TOYOT              P   
972356             14

In [50]:
tokenized_data = df['violation_description'].fillna("").apply(lambda x: x.split(" ")).tolist()
bm25 = BM25Okapi(tokenized_data)

In [51]:
test_queries = ["NO PARKING-DAY/TIME LIMITS", "FIRE HYDRANT", "DOUBLE PARKING"]
bm25_results = {}
for query in test_queries:
    tokenized_query = query.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    bm25_results[query] = pd.Series(doc_scores).nlargest(10)


In [55]:
def search_violations(query, df):
    weight_description_similarity = 0.6
    weight_recency = 0.4

    current_date = datetime.now()

    def recency_score(issue_date):
        delta = current_date - issue_date
        return 1 / (delta.days + 1)

    def description_score(description):
        if isinstance(description, str) and query in description:
            return 1
        else:
            return 0

    df['description_score'] = df['violation_description'].apply(description_score)
    df['recency_score'] = df['issue_date'].apply(recency_score)

    df['combined_score'] = (df['description_score'] * weight_description_similarity +
                            df['recency_score'] * weight_recency)

    results = df.sort_values(by='combined_score', ascending=False).head(10)

    results = results.drop(columns=['description_score', 'recency_score', 'combined_score'])

    return results


In [56]:
query = "NO PARKING-DAY/TIME LIMITS"
top_violations = search_violations(query, df)

print(top_violations[['summons_number', 'violation_description', 'issue_date']])


/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3165243962.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description_score'] = df['violation_description'].apply(description_score)
/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3165243962.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['recency_score'] = df['issue_date'].apply(recency_score)
/var/folders/f6/49sbsvhn5m56v71f8hbfhkk40000gn/T/ipykernel_38597/3165243962.py:20: SettingWithCopyWarning: 
A value

       summons_number violation_description issue_date
974348     1471648746                   NaN 2022-07-15
973526     1438863998                   NaN 2022-07-13
973993     1451845972                   NaN 2022-07-08
974006     1451852368                   NaN 2022-07-08
974060     1452081839                   NaN 2022-07-06
977728     1475858632                   NaN 2022-07-02
1766       1455210936                   NaN 2022-06-21
1434       1472443494                   NaN 2021-09-30
916        1472035276                   NaN 2021-08-09
977889     1472654134                   NaN 2021-08-06
